In [2]:
from scipy import stats
import math as m
from sklearn.neighbors.kde import KernelDensity
from sympy import *
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
import numpy as np
import pandas as pd 
from datetime import datetime as dt
import os

In [3]:
work_dir = os.getcwd()

In [42]:
def sharp_omega(freq):
    if freq == "weekly":
        df_etf = pd.read_csv(os.path.join(work_dir, "data", "output_weekly_with_rf.csv"), index_col = False)
    elif freq == "monthly":
        df_etf = pd.read_csv(os.path.join(work_dir, "data", "output_monthly_with_rf.csv"), index_col = False)
    else:
        raise print("Your freq is wrong!")
    Omega = pd.DataFrame()
    columns_name = []
    omega = []
    for i in range(1, df_etf.shape[1]-1):
        #exclude Date and Rf 
        columns_name.append(df_etf.columns[i])
        temp_etf = df_etf.iloc[:, i].dropna()
        x_value = np.linspace(-0.5,0.5,1000)
        Etf_gqussian = gaussian_kde(temp_etf)
        L = np.nanmean(df_etf['Rf'])
        pdf = (Etf_gqussian(x_value)/Etf_gqussian(x_value).sum())
        cdf = pdf.cumsum()
        cdf = pd.DataFrame(cdf, index=x_value)
        omega.append(((1-cdf).loc[L:].sum())/(cdf.loc[:L].sum()))
    Omega = pd.DataFrame(omega, index = columns_name)
    Omega.columns = ["Sharp-Omega"]
    Omega["Rank"] = Omega["Sharp-Omega"].rank(ascending=False) 
    return Omega

In [43]:
if __name__ == '__main__':
    Omega_weekly = sharp_omega(freq = "weekly")
    Omega_monthly = sharp_omega(freq = "monthly")

In [44]:
Omega_weekly

,Sharp-Omega,Rank
FPA,0.908515,29.0
FPXI,1.227656,3.0
IQDE,1.032982,23.0
IQDY,1.107187,16.0
CHIQ,1.157367,11.0
GEM,1.195171,5.0
ADRA,1.037718,22.0
PAF,1.028545,24.0
IEMG,1.202105,4.0
HEEM,1.057868,21.0
